<br>

# Introdução

Esse *script* tem como objetivo obter os dados sobre mananciais gerenciados pela [SABESP](http://site.sabesp.com.br), que abastecem a Região Metropolitana de São Paulo (RMSP). A gestão desses dados é feita pelo Laboratório de Sistemas de Suporte a Decisões em Engenharia Ambiental e de Recursos Hídricos ([LabSid](http://www.labsid.eng.br)), que dá suporte ao Sistema de Suporte a Decisões [SSD](http://ssd3sabesp.labsid.eng.br), bem como publiciza os dados no [*site* da SABESP](http://mananciais.sabesp.com.br/HistoricoSistemas).

<br>

O *script* foi desenvolvido a partir da interpretação do arquivo *json* que alimenta o *site*, identificado inspecionando a página. Funções específicas foram criadas para obter cada conjunto de dados (conforme o arquivo *json* se encontra separado). Ao final, para obtenção dos resultados, as funções são aplicadas em conjunto, otendo-se os dados para um intervalo entre datas, ou ainda, uma data única, com a finalidade de uma atualização diária.
![json](https://i.imgur.com/nvlAMuz.png)

<br>

***TODO***: Criar *script* para consumir esses dados:
- http://mananciais.sabesp.com.br/api/Mananciais/ResumoTelemetricos/2020-04-01

***TODO***: O Presente código tem a finalidade de obter os dados unicamente do Sistema Cantareira e, portanto, não se pensou em adicionar a possibilidade de obter os dados dos outros sistemas produtores, por meio da inclusão e ajuste desse parâmetro definido e fixado como *0*.

Inicialmente importa-se as bibliotecas que serão necessárias para rodar os códigos abaixo.

In [220]:
import os
import json
import time
import zlib
import requests
import calendar
import pandas as pd
import urllib.request
from datetime import date, datetime

<br>

E é criada a estrutura de pastas que será utilizada para armazenar os arquivos que serão criados ao longo do processo. Em específico, a pasta *data* é a que armazenará a tabela com as informações.

In [221]:
data_path = os.path.join('..', 'data')
docs_path = os.path.join('..', 'docs')

os.makedirs(data_path, exist_ok=True)
os.makedirs(docs_path, exist_ok=True)

<br>

# Série Histórica e Datas

A função abaixo retorna três parâmetros:
1. Tabela com uma série histórica, com uma data de início e uma data de término.
2. Data inicial em formato texto, para ser usada no nome do arquivo que vamos criar.
3. Data final em formato texto, para ser usada no nome do arquivo que vamos criar.

Caso não seja definida uma data final, será usado a data de hoje.
Importante ter atenção a isso pois a SABESP disponibiliza atualizações dos dados as 9:00 (conforme site deles) e, caso o presente código seja rodado entre a meia noite e as 9:00, tentar-se-á obter informações não disponíveis, provavelmente gerando erros. Para contornar isso, é necessário inserir, pelo menos, um parâmetro como *dia de hoje - 1*, por exemplo.

In [222]:
def create_df(start=date(1970, 1, 1), end=None):
    """
    Function to create date table, with only on colum named 'Data' as index.
    With no 'end' parameter is passed, the function will return a table until today
    With no 'start' parameter is passed, the function will return a table staring in firts day of de 70's.
    
    The function return two more parameters to used in filenames:
    filename_start > first day of table 
    filename_end   > last day of table as string    
    """    
    
    if end is None:
        end=date.today()
        
    else:
        pass
    
    # Dataframe to get dates
    df = pd.DataFrame(
        pd.date_range(pd.to_datetime(start), end=end),
        columns=['Data']
    ).set_index('Data')

    return (
        df,
        start.strftime('%Y.%m.%d'),
        end.strftime('%Y.%m.%d'),
    )

In [248]:
# Parameters
start = date(2021, 2, 1)
end = date(2021, 8, 1)

# Function
df_day, filename_start, filename_end = create_df(start, end)

# Results
print(filename_start)
print(filename_end)
print(df_day.index)

2021.02.01
2021.08.01
DatetimeIndex(['2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04',
               '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08',
               '2021-02-09', '2021-02-10',
               ...
               '2021-07-23', '2021-07-24', '2021-07-25', '2021-07-26',
               '2021-07-27', '2021-07-28', '2021-07-29', '2021-07-30',
               '2021-07-31', '2021-08-01'],
              dtype='datetime64[ns]', name='Data', length=182, freq=None)


<br>

# *Link* para fazer o *download* do Json

Inicialmente foram feitas tentativas diversas para melhor conhecimento do [*site* da SABESP](http://mananciais.sabesp.com.br/HistoricoSistemas?SistemaId=0) que disponibiliza as informações dos mananciais. Inicialmente tentou-se obter os dados pela técnica de *web scrapping*, até que descobriu-se que os dados são distribuídos por meio de uma API do SSD (Sistema de Suporte a Decisões).

Observou-se que a consulta manual apresenta os dados do mês da presente data até o primeiro dia do mês anterior. Por exemplo, se estamos no dia 25.**03**.2020, os dados apresentados serão dessa data até o dia 01.**02**.2020, retornando aproximadamente dados de 55 dias (30 dias de um mês hipotético e 25 do outro). O mesmo padrão irá ocorrer caso a consulta seja feita em 01.**03**.2020, a qual retornar-a os dados *até o primeiro dia do mês anterior*, ou seja, 01.**02**.2020.

Essa forma de "entregar" os dados foi considerada na requisição de dados pela API, a qual foi feita com uso do [urllib.request](https://stackoverflow.com/questions/32795460/loading-json-object-in-python-using-urllib-request-and-json-modules). A API tem seu *link* padrão apresentado abaixo, sendo inserido apenas duas variáveis: a data e o Sistema (aqui representado pelo 0 no final, que representa o Sistema Produtor Cantareira).

- http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/2020-03-25/0

In [224]:
url = 'http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/{}/{}/0'.format(
    start.strftime('%Y-%m-%d'),
    end.strftime('%Y-%m-%d')
)
url

'http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/2021-02-01/2021-08-01/0'

<br>

# Convertendo Json para tabela e extraíndo dados

Com o *link* da API com uma data definida, criou-se uma função para obter os dados em formato *json*, bastando inserir o site.

Em fevereiro de 2022 fui estudar por que que o site não mais funcionada. Havia detectado isso fazia algum tempo. Notei que o primeiro erro dizia respeito ao certificado SSL. Ainda assim não tinha sucesso. Descobri que a requisição retornava um gzip, que precisava ser descompactado! [Ref](https://stackoverflow.com/questions/53934881/getting-bytes-response-from-request)
```
URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)>
``` 
```python
import zlib

r = requests.get(url, verify=False)
decompressed_data=zlib.decompress(r.content, 16+zlib.MAX_WBITS)
data = json.loads(decompressed_data)
```

In [225]:
def get_json(url):
    # Get Array with data
    #webURL = urllib.request.urlopen(url)
    #my_bytes = webURL.read()

    # Transform Array into JSON
    #my_json = my_bytes.decode('utf8')
    #data = json.loads(my_json)  
    
    r = requests.get(url, verify=False)
    decompressed_data=zlib.decompress(r.content, 16+zlib.MAX_WBITS)
    data = json.loads(decompressed_data)
    return json.dumps(data, indent=2, sort_keys=True)

In [226]:
jsn = get_json(url)
#jsn

/home/michel/Documents/Conda/envs/pablocarreira-py39/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mananciais.sabesp.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Com o arquivo *json* contendo todos os dados do periodo selecionado, com diversas chaves e subchaves, iniciou-se a segmentação do arquivo, convertendo o arquivo para uma tabela, com a qual tenho mais familiaridade para editar e filtrar.

A função abaixo tem esse objetivo e já aproveita para excluir duas colunas que, aparentemente, não agregam informações.

In [227]:
def json2df(jsn):
    # Create dataframe
    df = pd.read_json(jsn)

    # Delete columns
    return df.drop(['FlagHasError', 'Message'], axis=1)

In [228]:
df = json2df(jsn)
df

,ReturnObj
DataFinal,01/08/2021
DataInicial,01/02/2021
ListaDados,"[{'Dados': [{'Chuva': 0.8, 'ComponenteId': 1, ..."
ListaDadosEspecial,"[{'Dados': [{'Chuva': None, 'Data': '2021-07-3..."
ListaDadosLocais,"[{'Dados': [{'Abreviatura': 'F-25bT', 'Compone..."
ListaDadosSistema,"[{'Data': '2021-02-01T00:00:00', 'objQETA': [{..."
ListaETAs,"[{'ComponenteId': 22, 'Nome': 'ETA Guaraú'}]"
ListaEspecial,"[{'Altitude': None, 'AreaDrenagem': None, 'Cod..."
ListaLocais,"[{'Abreviatura': 'F-25bT', 'ComponenteId': Non..."
ListaManobras,"[{'AberturaComporta': 26.3, 'ComponenteId': 4,..."


<br>

## Sistema Produtor (*json: SistemaId*)

Por meio do campo *SistemaId* é possível obter o código que define qual é o sistema produtor de água. Contudo, considerando que o presente *script* visa obter somente os dados do Sistema Cantareira, **tal função não será aplicada**.

In [229]:
def get_system(df):
    # JSON to dataframe    
    data = df.loc['SistemaId']['ReturnObj']
    
    # Results
    return data

In [230]:
get_system(df)

0

<br>

## Data Final  (*json: DataFinal*)

Identifica a data final empregada na API. Não se vislumbra muita utilidade para essa informação nesse momento, tendo em vista que foi o usuário que definiu esse parâmetro na definição do *link* de acesso à API. Logo, **tal função não será aplicada no código final**.

In [231]:
def get_enddate(df):
    # JSON to dataframe    
    data = df.loc['DataFinal']['ReturnObj']   
   
    # Results
    return datetime.strptime(data, '%d/%m/%Y').date()

In [232]:
data = get_enddate(df)
data

datetime.date(2021, 8, 1)

<br>

## Data Inicial  (*json: DataInicial*)

Identifica a data inicial empregada na API. Não se vislumbra muita utilidade para essa informação nesse momento, tendo em vista que foi o usuário que definiu esse parâmetro na definição do *link* de acesso à API. Logo, **tal função não será aplicada no código final**.

In [233]:
def get_startdate(df):
    # JSON to dataframe    
    data = df.loc['DataInicial']['ReturnObj']   
    
    # Results
    return datetime.strptime(data, '%d/%m/%Y').date()

In [234]:
data = get_startdate(df)
data

datetime.date(2021, 2, 1)

<br>

## Manobras Operacionais (*json: ListaManobras*)

Identifica todas as manobras listadas no site da SABESP. São dados mais descritivos, disponibilizados visando dar mais transparência a cadeia de comando para abrir ou fechar os reservatórios. Nesse primeiro momento tais dados não serão analisado e, portanto, **tal função não será aplicada**.

In [235]:
def get_manobras(df):
    # JSON to dataframe    
    lst = df.loc['ListaManobras']['ReturnObj']
    
    # Results
    return pd.json_normalize(lst)

In [245]:
df_manobras = get_manobras(df)
df_manobras.head()

,AberturaComporta,ComponenteId,ComponenteNome,Data,Observacao,OperacaoMotivoId,OperacaoMotivoNome,OperacaoTipoId,OperacaoTipoNome,RegistroManobraId,SSD2Id,SolicitanteId,SolicitanteNome,VazaoFundo,VazaoVertedor
0,26.3,4,Represa Atibainha,2021-07-29T09:29:00,"BRA - Reduzida a vazão de Jusante para 3,50m³/...",2,Solicitação do órgão gestor,1,Alteração de vazão,25125,None,3,DAEE,3.5,None
1,26.5,3,Represa Cachoeira,2021-06-18T05:00:00,"BRC - Aumentada a vazão de jusante para 4,5 m³...",2,Solicitação do órgão gestor,1,Alteração de vazão,24693,None,3,DAEE,4.5,None
2,6.0,1,Represa Jaguari/Jacareí,2021-07-29T09:30:00,Jusante BJA/BJC - Reduzida a vazão de Jusante ...,2,Solicitação do órgão gestor,1,Alteração de vazão,25124,None,3,DAEE,1.0,None


<br>

## Componentes do Sistema

<br>

### Reservatórios (*json: ListaRepresas*)

A função abaixo lista os reservatórios (ou represas) do Sistema Cantareira e outras que estão integradas, de alguma maneira, ao Sistema, inserindo também o identificador de cada reservatório (*ComponenteId*).

Apesar de trata-se de uma tabela que não retorna dados temporais (por exemplo: vazão, volume e chuva), ou seja, que variam ao longo do tempo, é fundamental para rotular de qual reservatório que são os dados temporais que serão obtidos nas próximas funções, visto que eles se valem, majoritariamente, do campo *ComponenteId*.

In [237]:
def list_represas(df):
    # JSON to dataframe
    lst = df.loc['ListaRepresas']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['temChuva','temNivel', 'temQjus', 'temQnat', 'temVolume'], axis=1)

    # Results
    return df

In [244]:
df_represas = list_represas(df)
df_represas.head()

,ComponenteId,Nome
0,1,Represa Jaguari/Jacareí
1,3,Represa Cachoeira
2,4,Represa Atibainha
3,5,Represa Paiva Castro
4,6,Represa Águas Claras


<br>

### Estruturas (Túneis e outros Pontos de Medição)  (*json: ListaLocais*)

A função abaixo lista os túneis e estações de monitoramento do Sistema Cantareira e outras que estão integradas, de alguma maneira, ao Sistema, inserindo também o identificador de cada local (*ComponenteId*).

Apesar de trata-se de uma tabela que não retorna dados temporais (por exemplo: vazão, volume e chuva), ou seja, que variam ao longo do tempo, contudo é fundamental para rotular de qual estrutura que são os dados temporais que serão obtidos nas próximas funções, visto que eles se valem, majoritariamente, do campo *ComponenteId* ou *abreviatura*.

In [261]:
def list_estruturas(df):
    # JSON to dataframe
    lst = df.loc['ListaLocais']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df.drop(
        ['Maximo','Minimo',
         'Data','Dia',
         'Valor','Unidade'],
        axis=1,
        inplace=True,
    )

    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('ComponenteId')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Results
    return df

In [262]:
df_estruturas = list_estruturas(df)
df_estruturas.head()

,ComponenteId,Abreviatura,LocalMedicaoId,Nome,SistemaId
0,NaN,F-25bT,46,F-25bT,0
1,7.0,Q T5,10,Q Túnel 5,0
2,8.0,Q T6,9,Q Túnel 6,0
3,9.0,Q T7,7,Q Túnel 7,0
4,36.0,Q ESI,11,Q Elev. Santa Inês,0


<br>

## Dados Diários

Nessa seção que serão obtidos diversos dados relevantes na operação do Sistema Cantareira, tais como:
- Vazão natural em cada reservatório;
- Vazão afluente em cada reservatório;
- Vazão defluente em cada reservatório;
- Nível e Volume de cada reservatório;
- Dados de Precipitação de cada reservatório.

Inicialmente, definiu-se uma função para renomear *strings*, visto que estas constarão nos cabeçalhos das tabelas a serem criadas. Aplicou-se a função na *df_represas* (criada acima) apenas para observar quais serão os nomes que constarão no cabeçalho das tabelas.

In [241]:
def rename_field(x):
    return(x.replace('/', '-').
           replace(' (', '-').
           replace('-', '_').
           replace(')', '').
           replace('Cesp', 'CESP').
           replace('Represa ', '').
           replace(' ', '')
          )

<br>

### Volume, QJusante e Chuva (*json: ListaDados*)

Extraíndo os dados do json, por meio da chave *ListaDados* e subchave *Dados*, foi obtido os dados de volume, vazão defluente e precpitação de cada reservatório.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome do reservatório. Portanto, foi necessário filtrar essas tabelas por reservatório, ajustar o cabeçalho inserindo o nome do reservatório em questão e, posteriromente, fazer um join das tabelas pelo campo *data*.

In [144]:
def list_volumes(df):
    # JSON to dataframe
    lst = df.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')

    # Define pivot to create new tables
    fields = df['Nome']
    fields = sorted(list(set(fields)))

    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = create_df()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        df_name = 'df_dados_{}'.format(j)
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[df_name] = df[df['Nome'] == i]

        # Deleta colunas
        locals()[df_name] = locals()[df_name].drop(['FlagConsolidado',
                                                    'NAMaxMax','NAMinMin',
                                                    'QJusanteMax','QJusanteMin',
                                                    'NivelUltimoDia',
                                                    'SistemaId','ComponenteId',
                                                    'UltimoDia',
                                                    'VazaoJusantePrincipal','VazaoJusanteSecundaria',
                                                    'VolumeOperacionalUltimoDia','VolumePorcentagemUltimoDia',
                                                    'VolumeTotalUltimoDia','Nome'], axis=1)

        # Renomeia as colunas
        locals()[df_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[df_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[df_name]['Data'] = pd.to_datetime(locals()[df_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full, locals()[df_name], on='Data', how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    return df_full

In [242]:
df_volumes = list_volumes(df)
df_volumes.head()

,Atibainha_Chuva,Atibainha_Nivel,Atibainha_QJusante,Atibainha_Volume,Atibainha_VolumeMaximo,Atibainha_VolumeMinimo,Atibainha_VolumeOperacional,Atibainha_VolumePorcentagem,Atibainha_VolumeTotal,Cachoeira_Chuva,...,PaivaCastro_VolumeTotal,ÁguasClaras_Chuva,ÁguasClaras_Nivel,ÁguasClaras_QJusante,ÁguasClaras_Volume,ÁguasClaras_VolumeMaximo,ÁguasClaras_VolumeMinimo,ÁguasClaras_VolumeOperacional,ÁguasClaras_VolumePorcentagem,ÁguasClaras_VolumeTotal
Data,,,,,,,,,,,,,,,,,,,,,
2021-02-01,0.2,783.33,1.96,226.103111,295.456327,199.204147,26.898964,27.946343,226.103111,17.4,...,27.984246,0.0,859.66,0.0,0.925554,1.028139,0.513304,0.412249,80.074099,0.925554
2021-02-02,0.0,783.35,2.96,226.485640,295.456327,199.204147,27.281493,28.343766,226.485640,0.4,...,27.984246,1.4,859.79,0.0,0.945215,1.028139,0.513304,0.431911,83.893081,0.945215
2021-02-03,0.8,783.35,3.00,226.485640,295.456327,199.204147,27.281493,28.343766,226.485640,27.8,...,28.066464,0.2,860.04,0.0,0.983765,1.028139,0.513304,0.470461,91.380952,0.983765
2021-02-04,0.0,783.36,3.00,226.677021,295.456327,199.204147,27.472874,28.542599,226.677021,0.0,...,28.190087,5.2,859.31,0.0,0.873883,1.028139,0.513304,0.360579,70.037804,0.873883
2021-02-05,16.8,783.40,3.00,227.443329,295.456327,199.204147,28.239182,29.338745,227.443329,5.0,...,28.231373,46.2,859.78,0.0,0.943693,1.028139,0.513304,0.430389,83.597522,0.943693


### Vazão Afluente e Vazão Natural  (*json: ListaDados*)

Extraíndo os dados do json, por meio da chave *ListaDados* e subchave *Qnat*, foram obtido os dados de vazão afluente e vazão naturalde cada reservatório.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome do reservatório. Portanto, foi necessário filtrar essas tabelas por reservatório, ajustar o cabeçalho inserindo o nome do reservatório em questão e, posteriromente, fazer um join das tabelas pelo campo *data*.

In [146]:
def list_vazao(df):
    # Represas
    df_represas = list_represas(df)
    
    # JSON to dataframe
    lst = df.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst, 'Qnat')

    # Merge Tables
    df = pd.merge(df, df_represas, on='ComponenteId', how='outer')

    # Define pivot to create new tables
    fields = df['Nome']
    fields = sorted(list(set(fields)))
    
    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = create_df()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        df_name = 'df_vazao'+'_'+j
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[df_name] = df[df['Nome'] == i]

        # Deleta colunas
        locals()[df_name] = locals()[df_name].drop(['ComponenteId','Nome',
                                                    'VazaoAfluenteMax','VazaoAfluenteMin',
                                                    'VazaoNaturalMax','VazaoNaturalMin'], axis=1)

        # Renomeia as colunas
        locals()[df_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[df_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[df_name]['Data'] = pd.to_datetime(locals()[df_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full,locals()[df_name], on='Data', how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    return df_full

In [249]:
df_vazao = list_vazao(df)
df_vazao.head()

,Atibainha_VazaoAfluente,Atibainha_VazaoNatural,Cachoeira_VazaoAfluente,Cachoeira_VazaoNatural,Jaguari_PBS_VazaoAfluente,Jaguari_PBS_VazaoNatural,Jaguari_Jacareí_VazaoAfluente,Jaguari_Jacareí_VazaoNatural,PaivaCastro_VazaoAfluente,PaivaCastro_VazaoNatural,ÁguasClaras_VazaoAfluente,ÁguasClaras_VazaoNatural
Data,,,,,,,,,,,,
2021-02-01,23.518519,4.910519,3.513802,3.513802,45.713959,45.713959,17.049313,17.049313,23.032000,2.801000,23.153507,0.221507
2021-02-02,27.614414,4.524414,4.948570,4.948570,40.979486,40.979486,21.700712,21.700712,24.048000,3.821000,22.991203,-0.956797
2021-02-03,23.246000,5.358000,8.231281,8.231281,50.476340,50.476340,21.758588,21.758588,24.491604,4.245604,22.231453,-1.208547
2021-02-04,24.705064,3.612064,4.151411,4.151411,29.720233,29.720233,21.796485,21.796485,21.559816,2.069816,22.385898,2.356898
2021-02-05,29.529303,6.955303,6.544344,6.544344,31.378090,31.378090,17.664486,17.664486,24.779850,7.119850,22.730118,-1.471882


<br>

### Sistema Equivalente  (*json: ListaDados*)

Extraíndo os dados do *json*, por meio da chave *ListaDados*, foram obtido os dados do Sistema Equivalente.
A SABESP entendo que o Sistema Equivalente é a somatária das represas que estão na Bacia do rio Piracicaba, ou seja:
- Jaguari/Jacareí;
- Cachoeira;
- Atibainha.

In [148]:
def list_SE(df):
    # JSON to dataframe
    lst = df.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['Dados','Data','Qnat'], axis=1)

    # Transform columns to list and reorder list
    col = df.columns.to_list()

    # Functions to rename
    col = ['SE_{}'.format(x) for x in col]
    col = [x.replace('SistemaEquivalente.', '').replace('SE_Data', 'Data') for x in col]

    # Rename Columns
    df.columns = col

    # Convert Data Column (object) to datatime colum
    df['Data'] = pd.to_datetime(df['Data'])

    # Results
    df = df.set_index('Data')
    df.dropna(how='all', inplace=True)
    
    return df

In [250]:
df_SE = list_SE(df)
df_SE.head()

,SE_QJusante,SE_VArmaz,SE_VUtil,SE_VazaoNatural,SE_VolumePorcentagem
Data,,,,,
2021-02-01,5.43,415.812895,973.945085,25.473634,42.693669
2021-02-02,7.90,416.775970,973.945085,31.173696,42.792553
2021-02-03,8.00,417.652227,973.945085,35.347869,42.882523
2021-02-04,8.00,418.381440,973.945085,29.559959,42.957395
2021-02-05,8.00,419.548197,973.945085,31.164133,43.077192


<br>

### Sistema Cantareira (*json: ListaDadosSistema*)

In [150]:
def list_SC(df):
    # JSON to dataframe
    lst = df.loc['ListaDadosSistema']['ReturnObj']
    df = pd.json_normalize(lst)
    
    # Delete columns
    df = df.drop(['objSistema.SistemaId', 'objQETA', 'objSistema.Data'], axis=1, errors='ignore')
    
    # Transform columns to list and reorder list
    col = df.columns.to_list()

    # Functions to rename
    col = ['SC_{}'.format(x) for x in col]
    col = [x.replace('objSistema', '') for x in col]
    col = [x.replace('.', '') for x in col]
    col = [x.replace('SC_Data', 'Data') for x in col]
    
    # Rename Columns
    df.columns = col
    
    # Convert Data Column (object) to datatime colum
    df['Data'] = pd.to_datetime(df['Data'])

    # Results
    df = df.set_index('Data')
    df.dropna(how='all', inplace=True)

    # Results
    return df

In [251]:
df_SC = list_SC(df)
df_SC.head()

,SC_Precipitacao,SC_VariacaoVolumePorcentagem,SC_VazaoAfluente,SC_VazaoJusante,SC_VazaoNatural,SC_VazaoProduzida,SC_VazaoRetirada,SC_VolumeOperacionalHm3,SC_VolumePorcentagem,SC_VolumeTotalHm3
Data,,,,,,,,,,
2021-02-01,4.90,0.06761,36.375,5.53,28.275,23.38280,23.38107,418.89064,42.65376,930.29931
2021-02-02,5.00,0.10007,43.095,8.00,34.995,22.73804,22.76364,419.87337,42.75383,931.28204
2021-02-03,8.15,0.10152,42.633,8.10,39.593,21.77035,21.78527,420.87040,42.85535,932.27907
2021-02-04,0.00,0.07565,38.000,8.10,31.630,23.57634,23.65768,421.61335,42.93100,933.02202
2021-02-05,16.20,0.13012,46.284,8.10,38.284,21.98882,21.92213,422.89121,43.06112,934.29988


<br>

### Vazão dos Túneis e outros Pontos de Medição (*json: ListaDadosLocais*)

Extraíndo os dados do json, por meio da chave *ListaDadosLocais* e subchave *Dados*, foram obtido os dados de vazão dos túneis Q7, Q6 Q5 e outros.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome da estrutura. Portanto, foi necessário filtrar essas tabelas por estrutura, adicionando ao cabeçalho seu repectivo nome e, posteriormente, fazer um join das tabelas pelo campo *data*.

In [181]:
def list_vazaoestruturas(df):
    # JSON to dataframe
    lst = df.loc['ListaDadosLocais']['ReturnObj']
    # Descobri que deixou de funcionar pois a partir de 01.06.2021 teve falhas no Q"SC-PS"
    #df = pd.json_normalize(lst, 'Dados')
    list_d = []
    for parte1 in lst:
        for parte2 in parte1['Dados']:    
            if isinstance(parte2, dict):
                list_d.append(parte2)
    df = pd.DataFrame(list_d)

    # Define pivot to create new tables
    fields = df['Abreviatura']
    fields = sorted(list(set(fields)))

    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))
    col.append(col.pop(col.index('Unidade')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = create_df()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        df_name = 'df_dados_{}'.format(j)
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[df_name] = df[df['Abreviatura'] == i].copy()
        #locals()[df_name] = df.loc[:, 'Abreviatura' == i]

        # Deleta colunas
        locals()[df_name].drop(
            [
                'Maximo', 'Minimo', 'Dia',
                'Abreviatura', 'ComponenteId',
                'LocalMedicaoId', 'Nome', 'SistemaId'
            ],
            axis=1,
            inplace=True
        )

        # Renomeia as colunas
        locals()[df_name].columns = [x if x=='Data' else '{}_{}'.format(j, x) for x in locals()[df_name].columns]

        # Convert Data Column (object) to datatime colum
        #locals()[df_name]['Data'] = pd.to_datetime(locals()[df_name]['Data'])
        locals()[df_name].loc[:, 'Data'] = pd.to_datetime(locals()[df_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full,locals()[df_name], on='Data', how='left')

    # Results
    df_full.set_index('Data', inplace=True)
    df_full.dropna(how='all', inplace=True)
    
    return df_full

In [252]:
df_vazaoestruturas = list_vazaoestruturas(df)
df_vazaoestruturas.head()

,F_25bT_Valor,F_25bT_Unidade,QESI_Valor,QESI_Unidade,QPS_SC_Valor,QPS_SC_Unidade,QSC_PS_Valor,QSC_PS_Unidade,QT5_Valor,QT5_Unidade,QT6_Valor,QT6_Unidade,QT7_Valor,QT7_Unidade
Data,,,,,,,,,,,,,,
2021-02-01,13.588,m3/s,22.932,m3/s,8.10,m3/s,0.0,m3/s,20.231,m3/s,10.508,m3/s,0.0,m3/s
2021-02-02,23.507,m3/s,23.948,m3/s,8.10,m3/s,0.0,m3/s,20.227,m3/s,14.990,m3/s,0.0,m3/s
2021-02-03,20.817,m3/s,23.440,m3/s,3.04,m3/s,0.0,m3/s,20.246,m3/s,14.848,m3/s,0.0,m3/s
2021-02-04,22.687,m3/s,20.029,m3/s,6.37,m3/s,0.0,m3/s,19.490,m3/s,14.723,m3/s,0.0,m3/s
2021-02-05,18.019,m3/s,24.202,m3/s,8.00,m3/s,0.0,m3/s,17.660,m3/s,14.574,m3/s,0.0,m3/s


<br>

## Dados Horários

Referem-se a transposição da bacia do rio Paraíba do Sul para a bacia do rio Piracicaba, por meio da Estação Elevatória de Água Bruta (EEAB) Jaguari, que despeja água na represa Atibainha.

Tais dados não serão aqui considerados, visto que já se encontram discretizados em dado diário na tabela acima. Logo, **tal função não será aplicada**.

<br>

### Estrutura de Transposição (*json: ListaEspecial*)

In [183]:
def list_EEAB(df):
    # JSON to dataframe
    lst = df.loc['ListaEspecial']['ReturnObj']
    df = pd.json_normalize(lst)
    
    # Results
    return df

In [253]:
df_vazaoEEAB_pontos = list_EEAB(df)
df_vazaoEEAB_pontos.head()

,Altitude,AreaDrenagem,Codigo,CursoDaAguaId,CurvaChave,FlagAparecePublico,FlagAtivo,FlagPossuiChuva,FlagPossuiNivel,FlagVazaoVolume,...,PostoId,PrecipitacaoUltima,PrecipitacaoUltima_Inteira,Rede,RepresaId,RoteiroAcesso,SistemaId,SistemaNome,VazaoUltima,VazaoUltima_Inteira
0,None,None,SB-CA-EAB-PS,None,None,False,None,False,False,0,...,1575,None,--,11,None,None,0,Cantareira,None,--


<br>

### Vazão de Transposição  (*json: ListaDadosEspecial*)

Dados horários, transferência 

In [185]:
def list_vazao_EEAB(df):
    # JSON to dataframe
    lst = df.loc['ListaDadosEspecial']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')
    
    # Results
    return df

In [254]:
df_vazaoEEAB = list_vazao_EEAB(df)
df_vazaoEEAB.head()

,Chuva,Data,Nivel,PostoId,Vazao,Volume
0,None,2021-07-31T12:00:00,None,None,7.407,None
1,None,2021-07-31T12:10:00,None,None,7.602,None
2,None,2021-07-31T12:20:00,None,None,7.544,None
3,None,2021-07-31T12:30:00,None,None,7.673,None
4,None,2021-07-31T12:40:00,None,None,7.529,None


<br>

## Dados das ETAs

<br>

### Lista das ETAs

In [187]:
def list_etas(df):
    # JSON to dataframe
    lst = df.loc['ListaETAs']['ReturnObj']
    df = pd.json_normalize(lst)

    # Results
    return df

In [255]:
df_etas = list_etas(df)
df_etas.head()

,ComponenteId,Nome
0,22,ETA Guaraú


<br>

### Dados das ETA

In [189]:
def list_etas_dados(df):
    # JSON to dataframe
    lst = df.loc['ListaDadosSistema']['ReturnObj']
    df = pd.json_normalize(lst, 'objQETA')
    
    return df

In [257]:
df_etas_dados = list_etas_dados(df)
df_etas_dados.head()

,AnoMes,ComponenteId,Data,Dia,Nome,SistemaId,VazaoProduzida,VazaoProduzidaMaxima,VazaoProduzidaMedia,VazaoProduzidaMinima,VazaoRetirada,VazaoRetiradaMaxima,VazaoRetiradaMedia,VazaoRetiradaMinima
0,0001-01-01T00:00:00,22,2021-02-01T00:00:00,1,ETA Guaraú,0,23.38280,None,None,None,23.38107,None,None,None
1,0001-01-01T00:00:00,22,2021-02-02T00:00:00,2,ETA Guaraú,0,22.73804,None,None,None,22.76364,None,None,None
2,0001-01-01T00:00:00,22,2021-02-03T00:00:00,3,ETA Guaraú,0,21.77035,None,None,None,21.78527,None,None,None
3,0001-01-01T00:00:00,22,2021-02-04T00:00:00,4,ETA Guaraú,0,23.57634,None,None,None,23.65768,None,None,None
4,0001-01-01T00:00:00,22,2021-02-05T00:00:00,5,ETA Guaraú,0,21.98882,None,None,None,21.92213,None,None,None


<br>

# Resultado: Série Histórica do Sistema Cantareira

Com todas as funções definidas, é possível aplicar tais funções sequencialmente, visando criar uma série histórica com todos os dados do Sistema Cantareira.

In [191]:
# Define data de início e fim
start = date(2021, 1, 1)
end   = date(2021, 9, 30)
#end   = datetime.date.today().strftime('%Y-%m-%d')

# Roda a primeira função para pegar a tabela com datas
df_day, filename_start, filename_end = create_df(start, end)

# Years's List
list_year = df_day.index.year
list_year = list(set(list_year))
list_year = sorted(list_year, reverse=True)
print(list_year)

# Zera os Objetos
dfs_volumes         = []
dfs_vazao           = []
dfs_SE              = []
dfs_SC              = []
dfs_vazaoestruturas = []

# Function to loop
for y in list_year:
    # Tempo
    print('Início do ano {} as {}'.format(y, datetime.now().strftime('%H:%M:%S')))
    
    # Variáveis de Data
    firstdayyear   = date(y, 1, 1)
    lastdayyear    = date(y, 12, 31)
    today          = date.today()
    
    if today < lastdayyear:
        lastday = today
    elif today >= lastdayyear:
        lastday = lastdayyear
        
    # Url
    url = 'http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/{}/{}/0'.format(
        firstdayyear.strftime('%Y-%m-%d'),
        lastday.strftime('%Y-%m-%d')
    )
    
    # Get json
    jsn                 = get_json(url)
    time.sleep(3)
    df                  = json2df(jsn)
    
    # Data    
    system              = get_system(df)
    startdate           = get_startdate(df)
    enddate             = get_enddate(df)    
    df_manobras         = get_manobras(df)
    
    # Represas
    df_represas         = list_represas(df)
    
    # Dados
    df_volumes          = list_volumes(df)
    df_vazao            = list_vazao(df)
    df_SE               = list_SE(df)
    df_SC               = list_SC(df)
    df_vazaoestruturas  = list_vazaoestruturas(df)
    
    # Concat Data
    dfs_volumes.append(df_volumes)
    dfs_vazao.append(df_vazao)
    dfs_SE.append(df_SE)
    dfs_SC.append(df_SC)
    dfs_vazaoestruturas.append(df_vazaoestruturas)
    
    print('Fim')
    
# Time
print('Fim as {}'.format(datetime.now().strftime('%H:%M:%S')))

[2021]
Início do ano 2021 as 10:16:43


/home/michel/Documents/Conda/envs/pablocarreira-py39/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mananciais.sabesp.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Fim
Fim as 10:17:15


<br>

## Conctatena e une tabelas

In [192]:
# Concat Data
dfs_volumes         = pd.concat(dfs_volumes)
dfs_vazao           = pd.concat(dfs_vazao)
dfs_SE              = pd.concat(dfs_SE)
dfs_SC              = pd.concat(dfs_SC)
dfs_vazaoestruturas = pd.concat(dfs_vazaoestruturas)

# One Table
dfs_volumes = pd.concat([dfs_volumes, dfs_vazao], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_SE], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_SC], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_vazaoestruturas], axis=1)

# Merge
df = pd.merge(df_day, dfs_volumes, left_index=True, right_index=True, how='left')

# Rename
df.reset_index(inplace=True)
df.rename(str.lower, axis='columns', inplace=True)
df = df.rename(
    columns=lambda x: x
    .replace('á', 'a')
    .replace('é', 'e')
    .replace('í', 'i')
    .replace('ó', 'o')
    .replace('ú', 'u')
    .replace('ã', 'a')
    .replace('ç', 'c')
)

# Results
display(df)

,data,atibainha_chuva,atibainha_nivel,atibainha_qjusante,atibainha_volume,atibainha_volumemaximo,atibainha_volumeminimo,atibainha_volumeoperacional,atibainha_volumeporcentagem,atibainha_volumetotal,...,qps_sc_valor,qps_sc_unidade,qsc_ps_valor,qsc_ps_unidade,qt5_valor,qt5_unidade,qt6_valor,qt6_unidade,qt7_valor,qt7_unidade
0,2021-01-01,1.0,783.650,0.72,232.261090,295.456327,199.204147,33.056943,34.344098,232.261090,...,1.42,m3/s,0.0,m3/s,16.821,m3/s,1.271,m3/s,0.0,m3/s
1,2021-01-02,1.6,783.600,0.50,231.293629,295.456327,199.204147,32.089482,33.338966,231.293629,...,7.15,m3/s,0.0,m3/s,20.541,m3/s,1.273,m3/s,0.0,m3/s
2,2021-01-03,0.4,783.545,1.47,230.231679,295.456327,199.204147,31.027532,32.235667,230.231679,...,8.10,m3/s,0.0,m3/s,20.489,m3/s,1.274,m3/s,0.0,m3/s
3,2021-01-04,25.0,783.520,1.50,229.749756,295.456327,199.204147,30.545609,31.734979,229.749756,...,8.10,m3/s,0.0,m3/s,17.739,m3/s,1.273,m3/s,0.0,m3/s
4,2021-01-05,0.0,783.490,1.50,229.172094,295.456327,199.204147,29.967947,31.134824,229.172094,...,8.10,m3/s,0.0,m3/s,15.511,m3/s,1.271,m3/s,0.0,m3/s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2021-09-26,4.8,783.480,4.50,228.979697,295.456327,199.204147,29.775549,30.934935,228.979697,...,0.00,m3/s,NaN,NaN,19.033,m3/s,26.169,m3/s,30.0,m3/s
269,2021-09-27,0.0,783.490,4.50,229.172094,295.456327,199.204147,29.967947,31.134824,229.172094,...,0.00,m3/s,NaN,NaN,19.059,m3/s,24.898,m3/s,27.0,m3/s
270,2021-09-28,0.0,783.490,4.50,229.172094,295.456327,199.204147,29.967947,31.134824,229.172094,...,0.00,m3/s,NaN,NaN,19.000,m3/s,21.793,m3/s,26.0,m3/s
271,2021-09-29,0.0,783.459,4.50,228.575916,295.456327,199.204147,29.371769,30.515432,228.575916,...,0.00,m3/s,NaN,NaN,24.600,m3/s,21.420,m3/s,25.2,m3/s


<br>

## Export to CSV

In [194]:
# Export
df.dropna(how='all', inplace=True)

df.to_csv(
    os.path.join(data_path, 'tab_Cantareira_{}__{}.csv'.format(filename_start, filename_end)),
    index=False,
    header=True,
    encoding='UTF-8-SIG',
    sep=';',
    decimal=',',
    date_format='%d/%m/%Y',
)

<br>

# Export

In [258]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [259]:
input_filename = '1_get_data.ipynb'
input_filepath = os.path.join(os.getcwd(), input_filename)

In [260]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(input_filepath)

# Write to output html file
with open(os.path.join(os.getcwd(), '..', 'src', 'get_data.py'),  'w', encoding='utf-8') as f:
    f.write(body)